<h1><center>Tarea 1: Roja o Azul</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Ortega 
- Nombre de alumno 2: María José Zambrano


### **Link de repositorio de GitHub:** [https://github.com/mzambrano1/Lab-programacion-cientifica](https://github.com/mzambrano1/Lab-programacion-cientifica)






# Temas a tratar

- Programación orientada a objetos.
- Manejo de datos en arreglos/tensores con `Numpy`.
- Análisis exploratorio de datos con `Pandas`
- Preprocesamiento con `Scikit-learn`.
- Composición de preprocesadores con Pipelines.

## Reglas:

- Fecha de entrega: Descrita en U-cursos
- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquier material del curso que estimen conveniente.



# Descripción del tarea (LEER ESTO).

En el presente notebook se presentan dos problemas, uno de **Segmentación de clientes en tiendas de retail** y otro enfocado en el **Análisis de temperaturas**. Producto de la larga extensión que podría significar la realización de los dos problemas, para efectos de esta tarea deben escoger **SOLAMENTE UNA** sección para desarrollar. Si desea puede borrar la pregunta que no realizará.

<p align="center">
  <img src="https://media1.giphy.com/media/ikwWGan6JZMo7bbf5L/giphy.gif?cid=790b761140b82e1c3d6754790ef49d96e8406bdac2f36d6f&rid=giphy.gif&ct=g">
</p>




#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go


# Librerias utiles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder



Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


----

# 1 Segmentación de Clientes en Tienda de Retail 🛍️

In [2]:
HTML('<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>')

## 1.1 Cargar Dataset [Sin puntos]

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>


*Mr. Lepin Mora* quien es el gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas y si es posible extraer los diferenciar los tipos de clientes que posee el retail. 

Para esto, el área de ventas les entrega el archivo ```online_retail_II.xlsx``` con el que se les pide que cargue y visualicen algunas de las filas que componen el Dataset.

De la primera visualización de los datos, señale los atributos que componen el dataset.

In [3]:
try:  
  from google.colab import drive
  drive.mount('/content/drive')
except ModuleNotFoundError:
    import os
    path = os.getcwd()

"\ntry:  \n  from google.colab import drive\n  drive.mount('/content/drive')\nexcept ModuleNotFoundError:\n    import os\n    path = os.getcwd()\n"

In [4]:

df_2009 = pd.read_excel('./online_retail_II.xlsx',sheet_name=0)
df_2011 = pd.read_excel('./online_retail_II.xlsx',sheet_name=1)


Se juntan ambos datos datasets en un solo dataframe, de manera que sea más sencillo de trabajar:

In [5]:
df = df_2009.append(df_2011,ignore_index = True)

Se explora un extracto de los datos:

In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Se exploran las columnas:

In [7]:
for col in df.columns:
  print(col)

Invoice
StockCode
Description
Quantity
InvoiceDate
Price
Customer ID
Country


## 1.2 Análisis Exploratorio de Datos [1 Punto]

En base a la primera visualización del dataset, *Don Mora* le solicita que realicen un análisis exploratorio de los datos, para esto les da **total libertad** para la revisión. De la revisión, ustedes deben explicar potenciales anomalías visualizadas y señalar si existe la necesidad de realizar una limpieza de datos.


In [8]:
#tamaño del dataframe
df.shape

(1067371, 8)

In [9]:
#se ceuntan datos únicos
df.nunique()

Invoice        53628
StockCode       5305
Description     5698
Quantity        1057
InvoiceDate    47635
Price           2807
Customer ID     5942
Country           43
dtype: int64

In [10]:
df.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


In [11]:
#países de los que se tienen registro
df['Country'].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland', 'Saudi Arabia',
       'Czech Republic', 'European Community'], dtype=object)

In [12]:
#se cuentan los nulos
df.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

Exploración de valores "NaN":

In [13]:
df[df.isna().any(axis=1)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1066997,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
1066998,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
1066999,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
1067000,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [14]:
descrptions_nan = df[df['Description'].isnull()]

In [15]:
descrptions_nan 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1060783,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
1060787,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
1060793,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [16]:
descrptions_nan[descrptions_nan['Customer ID'].isnull()]


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1060783,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
1060787,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
1060793,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [17]:
df_group_invoice = df.groupby('Invoice')

In [18]:
df_group_client = df.groupby('Customer ID')

In [19]:
#agrupación por el número de factura
df_group_client.describe()

Quantity                           ... Price                
               count       mean           std  ...   50%     75%     max
Customer ID                                    ...                      
12346.0         48.0   1.083333  15309.385753  ...  7.49  7.4900  142.31
12347.0        253.0  12.988142     16.477614  ...  1.95  3.7500   12.75
12348.0         51.0  53.215686     48.700848  ...  0.55  1.2500   40.00
12349.0        180.0   8.994444      7.846643  ...  2.55  6.2750  300.00
12350.0         17.0  11.588235      4.345383  ...  1.65  2.1000   40.00
...              ...        ...           ...  ...   ...     ...     ...
18283.0        986.0   1.757606      1.525774  ...  1.65  2.0800   15.95
18284.0         29.0  17.000000     14.992855  ...  1.25  2.5500   50.00
18285.0         12.0  12.083333     21.760926  ...  3.75  8.0875   50.00
18286.0         70.0   8.457143      7.224527  ...  2.10  5.8250   50.00
18287.0        156.0  19.301282     14.882516  ...  1.65  2.5500   50.00

[5942 rows x 16 columns]

**Justifique:**

Luego de realizar la exploración inicial se encontraron algunas anomalías. Primero se tienen cantidades de negativas de algunos objetos comprados al igual que se tienen precios negativos, cosas que resultan extrañas considerando que son tiendas que venden productos. También se encontraron valores NaN en las descripciones y en los id de los clientes. Se comprobó que todas las filas con NaN en la descripción, también tienen el id del cliente en NaN.

## 1.3 Primeras Visualizaciones [0,5 Puntos]

### 1.3.1 Obtención de TOPs 

Sin considerar los comentarios realizados en la sección 1.2, se les pide obtener el **Top de 30 productos más vendidos** y el **Top de 30 productos que generan más ganancias** para la tienda de retail. Deben considerar todo el registro temporal presente en el dataset y entregar la información en un gráfico de barras (Utilice `plotly`). ¿Los artículos más vendidos son los mismos que generan más ganancias?, Comente.

In [20]:
df['ganancia'] = df['Quantity']*df['Price']

In [21]:
df_product_grouped = df.groupby('StockCode')

In [22]:
suma = df_product_grouped.sum().reset_index()
suma['StockCode'] = suma['StockCode'].astype(str)
suma = suma.sort_values('Quantity')

In [23]:
fig = px.bar(suma, x='StockCode', y='Quantity',title='Ventas de productos ordenados')
fig.show()

In [24]:
best_30 = suma.tail(30)

In [25]:
fig = px.bar(best_30 , x='StockCode', y='Quantity',title = 'Ventas de los 30 productos más vendidos')
fig.show()

In [26]:
suma = suma.sort_values('ganancia')

In [27]:
fig = px.bar(suma, x='StockCode', y='ganancia',title='Ganacias generadas por cada producto')
fig.show()

In [28]:
ganacia_best_30 = suma.tail(30)

In [29]:
fig = px.bar(ganacia_best_30, x='StockCode', y='ganancia',title='Ganacias generadas por los 30 productos de mayor ganancias')
fig.show()

In [30]:
best_30.merge(ganacia_best_30,on='StockCode')['StockCode']

0    85099F
1     22086
2     22386
3     20725
4     22197
5     84879
6    85099B
7    85123A
Name: StockCode, dtype: object

**Justifique:**

Se repiten solo 7 de los 30 productos de cada clasificación. Lo anterior puede explicarse bajo la hipótesis que, en primer lugar, no se considera el precio de costo de los artículos, por lo que los artículos más caros estarían favorecidos en el ranking de ganacias, mientras que los artículos más vendidos usualmente poseen un precio bajo o moderado.

### 1.3.2 Visualización del registro temporal 

El dueño del retail en su afán por saber más sobre los datos de su firma les solicita que grafiquen las ventas respecto al tiempo. Con esto les aclara que durante el día tienen muchas variaciones en sus ventas, por lo que les recomienda que consideren el registro temporal como `año-mes-día`. ¿Es posible observar datos extraños?, Comente lo que observa del gráfico.

**Hint**: Para realizar esta task generen una función (quizás en el futuro les sea necesario 😮).

In [31]:
df['fecha'] = pd.to_datetime(df['InvoiceDate'],format = '%Y%m%d').dt.date

In [32]:
grouped_date = df.groupby('fecha')

In [33]:
suma_fecha = grouped_date.sum().reset_index()

In [34]:
fig = px.line(suma_fecha, x="fecha", y="Quantity",title='Ventas por día')
fig.show()

In [35]:
fig = px.line(suma_fecha, x="fecha", y="ganancia",title='Ganacias por día')
fig.show()

**Justifique:**

En los gráficos se puede ver que hay datos extraños, se observan ganacias y cantidades de productos vendidos negativos, como también peaks muy altos (que se alejan mucho de la tendencia central).

## 1.4 Limpieza de Datos [1,5 Puntos]
Como pudo ver en 1.1, 1.2 y 1.3 los datos presentan valores "extraños", es por esto que se le solicita que realice una **función** que permita **limpiar el dataset**. Realice esta función en base observaciones propias y justifique su elección.

Una vez realizada la función, **grafique nuevamente** el registro de ventas en el tiempo. ¿Es posible ver cambios?, Comente que observa.

In [36]:
df = df.dropna()
df = df[(df[['Quantity','Price']] > 0).all(1)]

In [37]:
grouped_date = df.groupby('fecha')
suma_fecha2 = grouped_date.sum().reset_index()

**Justificación de la limpieza:**

Anteriormente se observó que existían productos con precios negativos, como también cantidades vendidaes negativas, lo cual no tiene sentido al momento de realizar ventas. Es por esto que se decide eliminar todos estos valores de modo que se logre tener unos datos más limpios.

También se nota la presencia de valores "NaN" los cuales pueden causar problemas al manejar los datos, por lo que también fueron eliminados.

### 1.4.1 Gráfico

In [38]:
fig = px.line(suma_fecha2, x="fecha", y="Quantity",title='Ventas por día')
fig.show()

In [39]:
fig = px.line(suma_fecha2, x="fecha", y="ganancia",title='Ganacias por día')
fig.show()

**Justifique:**

Luego de la limpieza, en los gráficos se puede ver que a pesar que hay menos datos no hay valores negativos, y en consecuencia se tiene un gráfico con más sentido. No se eliminaron los peaks debido a que a pesar que sean valores anómalos, no existe (aún) evidencia que indique que se trata de valores mal ingresados.

## 1.5 Creación de nuevas Caracteristicas [1.5 Puntos] 

El area de ventas le comenta que existe una metodologia especial para realizar la segmentación de clientes, esta recive como nombre **LRMFP** y cada una de sus siglas se define como:

- Length (L): Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- Recency (R): Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, t_{m-i+1}; t_{m} es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- Monetary (M): El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- Frequency (F): Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- Periodicity (P): Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

Diseñe una función que permita obtener las caracteristicas **LRMFP**. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto. 

Una referencia interesante es [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.


**NOTA:** Se contarán las visitas de un cliente según su número de boleta (i.e. si un cliente tiene dos boletas para una misma fecha, se considera como dos visitas independientes).

**NOTA 2:** Para el cálculo del periodicity, no se usará el valor de n como arbitrario, ya que en el documento entregado no está definido de esta forma. (n = número de valores de tiempo entre visitas de un cliente, número fijo y distinto para cada cliente).

In [40]:
def recency(df, n):
  '''
  Returns a DataFrame with Customer ID and its respective recency
  
  Parameters:
  ------------------
  df: Pandas DataFrame
    Must contain Invoice, StockCode, Quantity, Price, Customer ID, Country, ganancia, fecha

  n: int
    n of the recency definition  
  '''
  fecha_final = df['fecha'].max() 

  grouped = df.drop_duplicates(subset='Invoice').groupby('Customer ID') 

  Y  = pd.DataFrame(grouped['fecha'].nth(list(np.arange(-n,0)))) # Se obtienen las n últimas visitas del cliente
  Y = Y.reset_index().reset_index()
  Y['index'] = Y['index']%n # Se acota el indice convenientemente para hacer un pivoteo
  
  pivoted = pd.pivot(Y, index= 'Customer ID',values = 'fecha', columns = 'index' ) # la i-esima fila del data frame contiene las fechas de las visitas del i-esimo cliente 

  resta = fecha_final - pivoted
  resta = resta.fillna(datetime.timedelta(0))

  resta['recency'] = resta.sum(axis = 1)/n
  return pd.DataFrame(pd.to_numeric(resta['recency'].dt.days, downcast='integer'))

def periodicity(df):
  '''
  Returns a DataFrame with periodicity

  PARAMETERS
  -------------------------------------
  df: Pandas DataFrame
    Must contain Invoice, StockCode, Quantity, Price, Customer ID, Country, ganancia, fecha

  '''

  grouped = df.drop_duplicates(subset='Invoice').groupby('Customer ID')
  max_visit = grouped.count()['Invoice'].max()
  Dates = pd.DataFrame(grouped['fecha'].nth(list(np.arange(-max_visit,0)))).reset_index() # Se extraen todas las visitas por cliente

  # Operaciones sobre el dataset
  Dates_grouped = pd.DataFrame(Dates.groupby('Customer ID')['fecha']) 
  Dates_grouped[1] = Dates_grouped[1].apply(lambda x: x.sort_values().values[1:]-x.sort_values().values[:-1]) # Se ordenan las fechas y se obtiene la diferencia
  Dates_grouped[1] = Dates_grouped[1].apply(lambda x: list(map(lambda y: y.days, x))) # Se pasa la diferencia a numero de dias
  Dates_grouped['Periodicity'] =  Dates_grouped[1]
  Dates_grouped['Periodicity'] = Dates_grouped['Periodicity'].apply(lambda x: np.std(x)) # Se calcula la desviacion estandar
  Dates_grouped = Dates_grouped.rename({0: 'Customer ID'}, axis= 1)
  Dates_grouped = Dates_grouped.set_index('Customer ID')

  return Dates_grouped[['Periodicity']]


def LRMFP(df, n_recency = 1):
  '''
  Returns the LRMFP features
  
  PARAMETERS
  ---------------------
  df: Pandas DataFrame
    Must contain Invoice, StockCode, Quantity, Price, Customer ID, Country, ganancia, fecha

  n_recency: int, (default = 1)
    n of the recency function
  '''
  grouped = df.groupby('Customer ID')
  grouped_no_duplicates = df.drop_duplicates(subset='Invoice').groupby('Customer ID')
  
  Length = pd.DataFrame(pd.to_numeric((grouped['fecha'].max() - grouped['fecha'].min()).dt.days, downcast= 'integer'))
  Monetary = pd.DataFrame(grouped.sum()['ganancia']/grouped_no_duplicates.count()['Invoice'])
  Recency = recency(df, n_recency)
  Frequency = pd.DataFrame(grouped_no_duplicates.count()['Invoice'])
  Periodicity = periodicity(df)

  #Agrupacion de las caracteristicas
  LRMFP_DF = Length.merge(Monetary, on='Customer ID')
  LRMFP_DF = LRMFP_DF.merge(Recency, on='Customer ID')
  LRMFP_DF = LRMFP_DF.merge(Frequency, on='Customer ID')
  LRMFP_DF = LRMFP_DF.merge(Periodicity, on='Customer ID')

  LRMFP_DF = LRMFP_DF.rename({0: 'Monetary', 'fecha':'Length', 'Invoice':'Frequency'}, axis = 1)

  return LRMFP_DF



In [41]:
lrmfp = LRMFP(df)

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice



In [42]:
lrmfp.head()

,Length,Monetary,recency,Frequency,Periodicity
Customer ID,,,,,
12346.0,400,6463.038333,325,12,62.511487
12347.0,402,704.165000,2,8,17.638085
12348.0,363,403.880000,75,5,49.716069
12349.0,571,1107.172500,18,4,152.283362
12350.0,0,334.400000,310,1,NaN


**Ejemplo de resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

## 1.5.1 Visualización de información y Outliers [0,5]

Visualice las diferenes caracteristicas obtenidas y comente que observa de ellas. ¿Es posible observar valores anómalos?. Proponga un mecanísmo sencillo para solucionar este problema (NO LO IMPLEMENTE).

In [43]:
lrmfp.head()

,Length,Monetary,recency,Frequency,Periodicity
Customer ID,,,,,
12346.0,400,6463.038333,325,12,62.511487
12347.0,402,704.165000,2,8,17.638085
12348.0,363,403.880000,75,5,49.716069
12349.0,571,1107.172500,18,4,152.283362
12350.0,0,334.400000,310,1,NaN


In [44]:
#BOXPLOT

for i in lrmfp.columns:
  fig = px.box(lrmfp, x = i , points='all')
  fig.show()

In [45]:
### HISTOGRAMAS

for i in lrmfp.columns:
  
  fig = px.histogram(lrmfp, i)
  fig.show()

Es posible observar valores que se salen de las tendencias generales; por ejemplo, las variables length, periodicity se ven fuertemente afectadas por la gente que fue solo una vez a comprar, lo que genera una alta frecuencia para el cero. Por otro lado, se puede ver en la variable Monetary, que existen valores demasiado altos que se salen de todo ajuste, posiblemente debido a clientes que fueron pocas veces a comprar pero que gastaron altas sumas de dinero. 

Para solucionar lo anterior, lo más sencillo es filtrar
valores según las siguientes cotas inferior y superior respectivamente (búsquese como *lower and upper fence*): 

$$Q_1 - 1.5 * IQR \:\:\:  \text{y}  \:\:\:Q_3 + 1.5 * IQR$$

Con $Q_i$ el i-ésimo cuartil, y IQR el rango intercuartil, equivalente a la diferencia entre el tercer y primer cuartil.



### 1.5.1.1 Implementación del filtro de outliers [Bonus - 0,5 puntos]

Implemente el filtro de outliers como un [Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) de `Scikit-learn`.

In [46]:
  described = lrmfp.describe()
  Q1 = described.iloc[4]
  Q3 = described.iloc[6]
  IQR = Q3 - Q1
  lower_fence = Q1 - 1.5 * IQR
  upper_fence = Q3 + 1.5 * IQR

In [47]:
upper_fence

Length         1280.000000
Monetary        778.946250
recency         910.000000
Frequency        16.000000
Periodicity     159.134366
dtype: float64

In [48]:
def filtro(df):
  described = df.describe()
  Q1 = described.iloc[4]
  Q3 = described.iloc[6]
  IQR = Q3 - Q1
  lower_fence = Q1 - 1.5 * IQR
  upper_fence = Q3 + 1.5 * IQR

  filtered_df = df[(df[df.columns] > lower_fence).all(1) & (df[df.columns] < upper_fence).all(1)]
  return filtered_df

transformer = FunctionTransformer(filtro)


In [49]:
sin_out = transformer.fit_transform(lrmfp)
sin_out.describe()

,Length,Monetary,recency,Frequency,Periodicity
count,3427.000000,3427.000000,3427.000000,3427.000000,3427.000000
mean,337.495185,303.889377,159.568427,5.231106,41.685529
std,220.336306,152.657544,174.028403,3.388925,40.303395
min,0.000000,11.161333,0.000000,2.000000,0.000000
25%,135.000000,189.807556,23.500000,2.000000,0.000000
50%,343.000000,277.891818,73.000000,4.000000,34.062170
75%,529.000000,388.930000,266.000000,7.000000,66.821337
max,736.000000,777.414000,738.000000,15.000000,159.000000


In [50]:
for col in sin_out.columns:
  fig = px.histogram(sin_out, x = col)
  fig.show()

Se puede ver que después de definir el filtro anteriormente descrito, la visualización es relativamente mejor, en el sentido que no hay valores extremos que la entorpezcan.

## 1.6 Segmentación [1 Punto]



Finalmente *Don Mora* le pregunta si seria posible realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de Clusters para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.6.1 Estandarizar Caracteristicas

Construya una función que nos permita estandarizar los atributos generados anteriormente. Para esto recuerde que una estandarización basica es:

$$Z = \dfrac{x-\mu}{s}$$

Con esto buscamos que cada atributo tenga un $\mu$ igual a 0 y una $\sigma$ igual a 1 (compruebe esto).

In [51]:
lrmfp.head()

,Length,Monetary,recency,Frequency,Periodicity
Customer ID,,,,,
12346.0,400,6463.038333,325,12,62.511487
12347.0,402,704.165000,2,8,17.638085
12348.0,363,403.880000,75,5,49.716069
12349.0,571,1107.172500,18,4,152.283362
12350.0,0,334.400000,310,1,NaN


In [52]:
scaled_features = StandardScaler().fit_transform(lrmfp)
Scaled_lrmfp = pd.DataFrame(scaled_features)
Scaled_lrmfp.columns = lrmfp.columns
Scaled_lrmfp.index =  lrmfp.index

In [53]:
Scaled_lrmfp.head(2)

,Length,Monetary,recency,Frequency,Periodicity
Customer ID,,,,,
12346.0,0.488937,4.997097,0.592985,0.438998,0.360878
12347.0,0.496661,0.257158,-0.949988,0.131502,-0.572493


In [54]:
Scaled_lrmfp.describe()

,Length,Monetary,recency,Frequency,Periodicity
count,5.878000e+03,5.878000e+03,5.878000e+03,5.878000e+03,4.255000e+03
mean,-2.914383e-16,1.995965e-17,4.670945e-17,2.905671e-16,-9.715169e-16
std,1.000085e+00,1.000085e+00,1.000085e+00,1.000085e+00,1.000118e+00
min,-1.055810e+00,-3.199892e-01,-9.595418e-01,-4.066161e-01,-9.393675e-01
25%,-1.055810e+00,-1.729050e-01,-8.401167e-01,-4.066161e-01,-8.295571e-01
50%,-2.023374e-01,-8.778508e-02,-5.057263e-01,-2.528681e-01,-2.673757e-01
75%,9.214658e-01,2.374011e-02,8.509434e-01,5.462788e-02,4.505234e-01
max,1.794248e+00,6.900967e+01,2.565888e+00,3.011236e+01,6.351070e+00


**Comprobación:**

In [55]:
print("Promedio: \n",Scaled_lrmfp.mean(),"\n")
print("Desviación estandar: \n",Scaled_lrmfp.std())

Promedio: 
 Length        -2.914383e-16
Monetary       1.995965e-17
recency        4.670945e-17
Frequency      2.905671e-16
Periodicity   -9.715169e-16
dtype: float64 

Desviación estandar: 
 Length         1.000085
Monetary       1.000085
recency        1.000085
Frequency      1.000085
Periodicity    1.000118
dtype: float64


### 1.6.2 Clustering

Estandarizados los datos un compañero de trabajo les facilita un Código para evaluar la cantidad de cluster óptimos para separar los datos. (Otra forma seria revisando la Silhoutte).

En esta parte solo se le pedirá reemplazar la variable ```ccustomers``` por la del dataframe estandarizado.

In [56]:
# Usamos metodo del codo para escoger el k que mejor ajusta el cluster.
Scaled_lrmfp = Scaled_lrmfp.fillna(0.0)

elbow = pd.DataFrame(columns=['SSE','k'])
for k in range(1, 21):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(Scaled_lrmfp)
    elbow = elbow.append({'SSE': kmeans.inertia_, 'k': k}, ignore_index=True)

# Son ploteados los Resultados
fig = go.Figure(data=[
    go.Scatter(x = elbow.k, y=elbow.SSE, mode='lines'),
    go.Scatter(x = elbow.k, y=elbow.SSE, mode='markers')
])

fig.update_layout(title="Evaluación de los Clusters (Metodo del Codo):",
                 xaxis_title = "Numero de Clusters:",
                 yaxis_title = "SSE",
                 showlegend=False)
fig.show()

### 1.6.3 Segmentación de Clientes 🎁

En base al gráfico anterior escoja un valor que encuentren correcto. Para esto cambie el valor de "k" en el codigo de mas abajo.

In [57]:
k=5
model = KMeans(n_clusters=k, random_state=80)
model.fit(Scaled_lrmfp)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=80, tol=0.0001, verbose=0)

Genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique.


In [58]:
Scaled_lrmfp["Cluster"] = model.labels_

grouped_cluster = Scaled_lrmfp.groupby('Cluster')
mean = grouped_cluster.mean() 
count =  grouped_cluster.count()

mean['count'] = count['Length']
display(mean)

,Length,Monetary,recency,Frequency,Periodicity,count
Cluster,,,,,,
0,-0.801048,-0.039196,1.331379,-0.320080,-0.227231,1822
1,1.686667,0.454257,-0.848988,11.984741,-0.818646,21
2,-0.570186,-0.017913,-0.528064,-0.264789,-0.417599,1825
3,-0.264127,69.009673,-0.959542,-0.329742,-0.939367,1
4,1.115863,0.011569,-0.653357,0.368980,0.540635,2209


Se ve que hay diferencias en al menos una de las caracteristicas para cada cluster, lo que da un primer indicio que el clustering ha funcionado.  

Es posible notar que el cluster 3 es claramente un outlier, basta con ver la variable monetary y ver que se aleja demasiado de la tendencia central. Se hará un clustering esta vez sin outliers para comparar resultados.

### Clustering sin Outliers

In [59]:
scaled_features = StandardScaler().fit_transform(sin_out)
Scaled_sin_out = pd.DataFrame(scaled_features)
Scaled_sin_out.columns = sin_out.columns
Scaled_sin_out.index =  sin_out.index

In [60]:
# Usamos metodo del codo para escoger el k que mejor ajusta el cluster.
Scaled_sin_out = Scaled_sin_out.fillna(0.0)

elbow = pd.DataFrame(columns=['SSE','k'])
for k in range(1, 21):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(Scaled_sin_out)
    elbow = elbow.append({'SSE': kmeans.inertia_, 'k': k}, ignore_index=True)

# Son ploteados los Resultados
fig = go.Figure(data=[
    go.Scatter(x = elbow.k, y=elbow.SSE, mode='lines'),
    go.Scatter(x = elbow.k, y=elbow.SSE, mode='markers')
])

fig.update_layout(title="Evaluación de los Clusters (Metodo del Codo):",
                 xaxis_title = "Numero de Clusters:",
                 yaxis_title = "SSE",
                 showlegend=False)
fig.show()

In [63]:
k=4
model = KMeans(n_clusters=k, random_state=80)
model.fit(Scaled_sin_out)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=80, tol=0.0001, verbose=0)

In [64]:
Scaled_sin_out["Cluster"] = model.labels_

grouped_cluster = Scaled_sin_out.groupby('Cluster')
mean = grouped_cluster.mean() 
count =  grouped_cluster.count()

mean['count'] = count['Length']
display(mean)

,Length,Monetary,recency,Frequency,Periodicity,count
Cluster,,,,,,
0,1.036151,0.346870,-0.597290,1.612561,0.137145,702
1,-0.646860,-0.019225,-0.509330,-0.604371,-0.735239,1026
2,-0.855917,-0.248385,1.574865,-0.606653,-0.609036,792
3,0.677163,-0.029831,-0.336740,-0.034690,1.257373,907


**Justifique:**

Al igual que antes, se pueden notar características distintas para cada cluster. Sin embargo, y en contraste con el resultado de hacer clustering con los outliers, se obtien resultados más balanceados con caracteristicas más similares entre los clusters (clusters más cercanos entre sí).


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
|         | mean    | mean      | mean     | mean        | mean  | count |
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

In [65]:
!pip install umap-learn

In [66]:
import umap

In [67]:
umap_sin_out = umap.UMAP().fit_transform(Scaled_sin_out.drop('Cluster', axis=1))
umap_lrmfp = umap.UMAP().fit_transform(Scaled_lrmfp.drop('Cluster', axis=1))

In [68]:
Scaled_sin_out['x_umap'] = umap_sin_out[:, 0]
Scaled_sin_out['y_umap'] = umap_sin_out[:, 1]

px.scatter(
    Scaled_sin_out,
    x='x_umap',
    y='y_umap',
    color='Cluster',
)

In [69]:
Scaled_lrmfp['x_umap'] = umap_lrmfp[:, 0]
Scaled_lrmfp['y_umap'] = umap_lrmfp[:, 1]

px.scatter(
    Scaled_lrmfp,
    x='x_umap',
    y='y_umap',
    color='Cluster',
)

--------------------------------------------------------------------------------

# Cierre
Recuerden que la tarea tiene un plazo de entrega de dos semanas y que **los días de atraso son válidos para las tareas**. Cualquier duda de la tarea, no duden en contactarnos por mail o U-cursos.



<p align="center">
  <img src="https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif">
</p>